## Function Calling in OpenAI:  Cleaning Service

In [17]:
#!pip install openai
#!pip install python-dotenv

In [18]:
import json
import pandas as pd
import openai
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("openai_api_key")

client = openai.OpenAI()

This is a very similar "helper" function to the one we have been using.  This one includes the arguments tools and tool_choice, which is where we specify the function that we want the model to "call" in its response.

In [19]:
def get_output_service_category(messages_input, model_input="gpt-4o-mini", temperature_input = .5) :

    response = client.chat.completions.create(
        model = model_input,
        messages = messages_input,
        temperature = temperature_input,
        tools = [get_menu_tool],
        tool_choice = {"type": "function", "function": {"name": "Get_Custom_Menu"}}
    )
    
    response = response.choices[0].message
    return response

Create schema for function call.

The function Extract_customer_request_attributes has a name, description and 1 parameter called "service_category".

In [20]:
parameters = {
    "type": "object",
    "properties": {
        "service_category": {
            "type": "string",
            "enum": ["Residential", "Commercial", "Landscape"],
            "description": "The category of service that meets the customer's needs",
        }
    },
    "required": ["service_category"],
}

get_menu_tool = {
    "type": "function",
    "function": {
        "name": "Get_Custom_Menu",
        "description": "Retreives a menu of relevant services based on incoming customer request for home mainentance services",
        "parameters": parameters,
    }
}

In [21]:
system_message = f"""
You are OrderBot, an automated service to help customers sign up for the firm's cleaning services. \
You first greet the customer and find the service category of their inquiry. \
You wait to collect all the customer's needs, then summarize and check for a final time if the customer wants to add anything else. \
Make sure to clarify all options and extras to uniquely identify the item from the menu.\
You respond in a short, very conversational friendly style. \
"""

customer_inquiry = f"""
I would like to know more about your services.  I have a 3 bedroom house that I would like cleaned twice a month.
"""

context = [ {'role':'system', 'content': system_message},
            {'role':'user', 'content': customer_inquiry}]


response = get_output_service_category(context, temperature_input = 0)
response

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_6t78QY8Fca3dBbGVv8ao88be', function=Function(arguments='{"service_category":"Residential"}', name='Get_Custom_Menu'), type='function')], annotations=[])

Notice the completion to our prompt with function call has content = None and, instead, an object in the class ChatCompletionMessageToolCall in the tool_calls attribute.  This object contains the function arguments that the LLM created as the response to our prompt.

(In this example, the function argument is the category of service for the customer inquiry, which was extracted by the first call to the LLM.)

In [22]:
service_category_input = json.loads(response.tool_calls[0].function.arguments)['service_category']
service_category_input

'Residential'

We are reading in a .csv file which contains the services offered for each of the three possible service categories: Residential, Commercial and Landscape.

In [23]:
service_df = pd.read_csv('4_1_FunctionCalling_ServiceMenu.csv')
service_df = service_df.set_index('service_category')
service_df

,service_type,price
service_category,,
Residential,Standard Cleaning,$500 up to 3 bedrooms
Residential,Standard Cleaning,$100 each additional bedroom
Residential,Deep Cleaning,$75 Inside cabinets
Residential,Deep Cleaning,$50 Windows
Residential,Deep Cleaning,$150 Carpet steam clean
Residential,Eco-Friendly Cleaning,Additional $200 for environmentally friendly p...
Residential,Laundry Service,$25 per load for wash and dry
Residential,Laundry Service,$5 per item for ironing
Commercial,Office Cleaning,"$20 per sqare foot up to 10,000 square feet"


Create a dictionary from just the menu of services for the service category returned by the LLM in response to the customer inquiry.

In [24]:
def get_menu(service_category_input):
    
    menu = service_df.loc[service_category_input].groupby('service_type')['price'].apply(list).to_dict()

    return menu

In [25]:
get_menu(service_category_input)

{'Deep Cleaning': ['$75 Inside cabinets',
  '$50 Windows',
  '$150 Carpet steam clean'],
 'Eco-Friendly Cleaning': ['Additional $200 for environmentally friendly products and practices'],
 'Laundry Service': ['$25 per load for wash and dry',
  '$5 per item for ironing'],
 'Standard Cleaning': ['$500 up to 3 bedrooms',
  '$100 each additional bedroom']}

Now, let's put together a "system" with two calls to the LLM.

The first takes the customer inquiry and returns the service category of the request, in the form of an argument for the function get_service_category().   
After get_service_category(service_category_input) is run and the appropriate menu of serives retrieved from the db, a second LLM call is made with the menu appended as a new message.   
The second (and subsequent) LLM calls talk the customer through their choices based on the menu retrieved.

In [26]:
def initialize_chatbot(system_message, customer_inquiry):
    context = [ {'role':'system', 'content': system_message},
                {'role':'user', 'content': customer_inquiry}]
    response = get_output_service_category(context, temperature_input = 0)
    context.append({'role':'assistant', 'content':f"{response}"})
    return context, response

In [27]:
def get_output_from_messages(messages_input, model_input = "gpt-4o-mini", temperature_input = 0):
    response = client.chat.completions.create(
        model = model_input,
        messages = messages_input,
        temperature = temperature_input,
    )
    response = response.choices[0].message.content
    return response

In [28]:
def add_new_message(context, user_message):
    context.append({'role':'user', 'content':f"{user_message}"})
    response = get_output_from_messages(context, temperature_input = 1)
    context.append({'role':'assistant', 'content':f"{response}"})
    return context, response

In [29]:
system_message = f"""
You are OrderBot, an automated service to help customers sign up for the firm's cleaning services. \
You first greet the customer and find the service category of their inquiry. \
You wait to collect all the customer's needs, then summarize and check for a final time if the customer wants to add anything else. \
Make sure to clarify all options and extras to uniquely identify the item from the menu.\
You respond in a short, very conversational friendly style. \
"""

customer_inquiry = f"""
I would like to know more about your services.  I have a 3 bedroom house I would like cleaned.
"""

context, response = initialize_chatbot(system_message, customer_inquiry)
service_category_input = json.loads(response.tool_calls[0].function.arguments)['service_category']
menu_for_cust = get_menu(service_category_input)
context.append({'role':'function', 'name': 'Extract_customer_request_attributes', 'content':json.dumps(menu_for_cust)})
context, response = add_new_message(context, "")
print(response)


Hey there! It sounds like you're interested in our cleaning services for your 3-bedroom house. We have different packages you can choose from:

1. **Standard Cleaning** - This is $500 for up to 3 bedrooms. 
2. **Deep Cleaning** - This includes additional options like:
   - $75 for cleaning inside cabinets,
   - $50 for window cleaning,
   - $150 for carpet steam cleaning.
3. **Eco-Friendly Cleaning** - If you're looking for environmentally friendly products and practices, there's an additional charge of $200.
4. **Laundry Service** - We also offer laundry services at $25 per load for wash and dry, and $5 per item for ironing.

Do any of these options catch your eye? Or would you like to add anything else?


In [30]:
new_message = "The Depp Cleaning sounds perfect"
context, response = add_new_message(context, new_message)
print(response)

Great choice! The Deep Cleaning is a thorough service. Just to confirm, would you like to include any of the additional options, like cleaning inside cabinets, window cleaning, or carpet steam cleaning?  Let me know if you want any of those extras or any other services!


In [31]:
context

[{'role': 'system',
  'content': "\nYou are OrderBot, an automated service to help customers sign up for the firm's cleaning services. You first greet the customer and find the service category of their inquiry. You wait to collect all the customer's needs, then summarize and check for a final time if the customer wants to add anything else. Make sure to clarify all options and extras to uniquely identify the item from the menu.You respond in a short, very conversational friendly style. "},
 {'role': 'user',
  'content': '\nI would like to know more about your services.  I have a 3 bedroom house I would like cleaned.\n'},
 {'role': 'assistant',
  'content': 'ChatCompletionMessage(content=None, refusal=None, role=\'assistant\', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id=\'call_mweCQ2iTrBWBu4XrpT9T8qvL\', function=Function(arguments=\'{"service_category":"Residential"}\', name=\'Get_Custom_Menu\'), type=\'function\')], annotations=[])'},
 {'role': 'funct